# RAG application with Azure Open AI & Azure Cognitive Search
## Resume analysis usecase - 03 Search examples

### Objective
Let's build an application that will analyse **resume PDF documents using a RAG application.**

<img src="https://github.com/retkowsky/images/blob/master/HR.jpg?raw=true">

**Retrieval-Augmented Generation (RAG)** can exhibit variability in its implementation, but at a fundamental level, employing RAG within an AI-driven application involves the following sequential steps:

- The user submits a query or question.
- The system initiates a search for pertinent documents that hold the potential to address the user's query. These documents are often comprised of proprietary data and are maintained within a document index.
- The system formulates an instruction set for the Language Model (LLM) that encompasses the user's input, the identified relevant documents, and directives on how to utilize these documents to respond to the user's query effectively.
- The system transmits this comprehensive prompt to the Language Model.
- The Language Model processes the prompt and generates a response to the user's question, drawing upon the context provided. This response constitutes the output of our system.

### Steps
- Uploading PDF documents into an Azure Cognitive Search Index
- Use of some Azure Cognitive Search queries to get some answers
- Use a GPT model to analyse the answer (summmary, keywords generation)
- Get the text from the document and the reference to validate the proposed answer
- Chatbot experience using Azure Open AI to ask questions and get results provided by AI with references

### Process
<img src="https://github.com/retkowsky/images/blob/master/rag.png?raw=true" width=800>

In [1]:
import datetime
import gradio as gr
import langchain
import openai
import os
import sys
import time

from azure_rag import (
    delete_index,
    ask_gpt,
    index_status,
    index_stats,
    openai_text_embeddings,
    upload_docs,
    similarity_comparison,
    azure_openai,
    ask_your_own_data,
)
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

In [2]:
print(f"Today: {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today: 16-Nov-2023 11:47:57


In [3]:
print(f"Python version: {sys.version}")
print(f"OpenAI version: {openai.__version__}")
print(f"Langchain version: {langchain.__version__}")

Python version: 3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]
OpenAI version: 0.28.1
Langchain version: 0.0.300


## 1. Settings

In [4]:
load_dotenv("azure.env")

# Azure Open AI
openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

# Azure Cognitive Search
azure_cs_endpoint = os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT")
azure_cs_key = os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY")

In [5]:
# Azure Open AI models (Should be deployed on your Azure OpenAI studio)
embed_model: str = "text-embedding-ada-002"  # Embedding model
gptmodel: str = "gpt-35-turbo-16k"  # GPT Model

# Azure Cognitive search index
index_name: str = "resume-rag-demo"

In [6]:
print(f"Embedding model: {embed_model}\nAzure Open AI model: {gptmodel}")
print(f"We will use the Azure Cognitive Search index: {index_name}")

Embedding model: text-embedding-ada-002
Azure Open AI model: gpt-35-turbo-16k
We will use the Azure Cognitive Search index: resume-rag-demo


## 2. Azure Cognitive Search index status

Our index is available and ready to use

In [7]:
print(f"Azure Cognitive Search index = {index_name}")

Azure Cognitive Search index = resume-rag-demo


In [8]:
index_status(index_name, azure_cs_endpoint, azure_cs_key)

Azure Cognitive Search Index: resume-rag-demo 

{
     "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#indexes/$entity",
     "@odata.etag": "\"0x8DBE6962627FA4D\"",
     "name": "resume-rag-demo",
     "defaultScoringProfile": null,
     "fields": [
          {
               "name": "id",
               "type": "Edm.String",
               "searchable": false,
               "filterable": true,
               "retrievable": true,
               "sortable": false,
               "facetable": false,
               "key": true,
               "indexAnalyzer": null,
               "searchAnalyzer": null,
               "analyzer": null,
               "normalizer": null,
               "synonymMaps": []
          },
          {
               "name": "content",
               "type": "Edm.String",
               "searchable": true,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
               "facet

In [9]:
document_count, storage_size = index_stats(index_name, azure_cs_endpoint, azure_cs_key)

Azure Cognitive Search index status for: resume-rag-demo 

{
  "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#Microsoft.Azure.Search.V2021_04_30_Preview.IndexStatistics",
  "documentCount": 1509,
  "storageSize": 46394403
}


In [10]:
print(f"Number of documents in the index = {document_count}")
print(f"Size of the index = {round(storage_size / (1024 * 1024), 2)} MB")

Number of documents in the index = 1509
Size of the index = 44.25 MB


## 3. Search with Azure Open AI

### Testing

In [11]:
%%javascript Python
OutputArea.auto_scroll_threshold = 9999

<IPython.core.display.Javascript object>

This is to maximize the display output

We can do some quick tests

In [12]:
print(f"Azure OpenAI model to use: {gptmodel}")

Azure OpenAI model to use: gpt-35-turbo-16k


In [13]:
print(azure_openai("What time is it?", gptmodel))

I'm sorry, I don't have access to real-time information.


In [14]:
print(azure_openai("Who are you?", gptmodel))

I am a human resources expert.


In [15]:
print(azure_openai("Qui es-tu?", gptmodel))

Je suis un expert en ressources humaines.


In [16]:
print(azure_openai("What can you do for me?", gptmodel))

I can provide you with a summary of a resume, including key points, details, products/technologies mentioned, skills, companies/organizations worked for, schools/universities attended, and any other valuable information about the profile.


### Ask your own data 

We can ask any questions related with the documents we have processed. You can ask in any language.

In [19]:
# Embeddings engine
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(engine=embed_model)

In [20]:
# Azure Cognitive Search as the vector store
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=azure_cs_endpoint,
    azure_search_key=azure_cs_key,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [21]:
query = "Do we have some designer profiles with some great international experiences"

ask_your_own_data(query, vector_store, gptmodel, topn=3)

Profile 1:

1. Established Designer with a reputation for exquisitely designed collections, maximizing company profits and surpassing margin goals.
2. Experienced Designer with a strong track record of designing multiple lines for international and domestic departments, leading research and development of seasonal trends, and providing leadership in preparation of skus for line reviews and meetings with buyers. Also skilled in critical thinking, time management, adaptability, social perceptiveness, attention to detail, and innovation.
3. Products or technologies mentioned in the resume: PLM (Product Lifecycle Management)
4. Skills mentioned in the resume: Critical thinking, time management, communication, adaptability, social perceptiveness, attention to detail, innovation.
5. Companies or organizations where this profile has worked: CompanyName.
6. Schools or universities mentioned in the resume: The Fashion Institute of Design and Merchandising.
7. Other valuable aspects of this prof

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/DESIGNER/68240723.pdf


Profile 2:

1. Successful Design Consultant with over 26 years of experience in managing large-scale interior and exterior projects for residential homes, exhibiting sophisticated taste and a unique design style.
2. This Interior Designer has 26 years of experience in managing interior and exterior projects for residential homes, with a focus on low to high-end properties. They have a can-do attitude and valuable design insight gained from past creations. They excel in concept development, space planning, color and material application, and interior architectural detailing. They are proficient with furniture systems and have a strong leadership ability. They have successfully managed and supervised large-scale remodeling projects and have worked with Dr. Vettese for over 18 years. They have also introduced sustainable homes to third world countries and have experience in off-the-grid home construction. They have a degree in Arts from Roosevelt University and strong skills in budgets, 

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/DESIGNER/13518263.pdf


Profile 3:

1. This designer has experience in jewelry collection design for brides shows and is seeking a company that supports teamwork and utilizes employee abilities.
2. This designer has excellent teamworking skills, planning and organizing events, creativity in working, time management skills, and the ability to work under pressure. They also have experience in communicating and building relationships with people of varying backgrounds. They are computer literate and have fluency in MS Office and the World Wide Web. They have a Bachelor of Fine Arts in Visual Communication from American University in Dubai.
3. Products or technologies mentioned in the resume: MS Office (Word, Excel, PowerPoint, Access), World Wide Web (web search).
4. Skills mentioned in the resume: teamworking, planning and organizing events, creativity, time management, communication, leadership, computer literacy, typing skills.
5. Companies or organizations where this profile has worked: CompanyName (not spe

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/DESIGNER/76010167.pdf

In [22]:
query = (
    "I need some business development profiles with more than 10 years of experience"
)

ask_your_own_data(query, vector_store, gptmodel, topn=3)

Profile 1:

1. A business development, sales, and account management professional with 15+ years of experience working with a range of clients from SMB to national brands.
2. This candidate has a strong track record in cultivating innovative business concepts, creating value, and developing relationships that drive revenues and secure deals. They are skilled in B2B sales, negotiation, marketing promotions, sponsorship sales, and lead generation. They are also an effective communicator, experienced in collaborating with senior leadership teams and presenting to key stakeholders.
3. Products or technologies mentioned in the resume: Dell, GEMS, Access, Excel, Outlook, PowerPoint, Publisher, Word.
4. Skills mentioned in the resume: business development, cold-calling, consulting, content creation, client relations, marketing, market research, negotiation, networking, organizational skills, sales, strategy, teaching, web content.
5. Companies or organizations where this profile has worked: N

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/BUSINESS-DEVELOPMENT/20357858.pdf


Profile 2:

1. Results-driven and highly skilled Account Manager and Sales Leader with 10+ years of experience developing and executing plans that cover all aspects of the sales cycle.
2. This profile is a Director of Business Development with a proven track record in strategic account development, relationship selling, and strong deal closing. They have successfully increased annual sales by 67% and territory penetration by 25%. They have managed a portfolio of accounts totaling $10,500,000 in annual sales and $32,000,000 in total contract value. They have also generated a pipeline of over $150,000,000 in net new opportunities in 12 months. They have experience in the public sector industry, securing contracts for enterprise government software, SaaS, IaaS, and Managed Service solutions. They have also served as a Deputy Secretary of State, where they were accountable for the day-to-day operations of multiple divisions and implemented a new Statewide Voter Registration software solut

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/BUSINESS-DEVELOPMENT/10704573.pdf


Profile 3:

1. Experienced Business Development Manager with a strong track record in sales and marketing, strategic planning, and team leadership.
2. Accomplished Business Development Manager with over 20 years of experience in sales and marketing, successfully improving sales and growing the customer base. Skilled in strategic planning, program management, and effective communication. Proven ability to build relationships with key decision-makers and identify valuable business opportunities.
3. Products or technologies mentioned in the resume: chip assembly equipment, design management and collaboration software, precious and non-precious metals puttering targets and evaporation materials.
4. Skills mentioned in the resume: key decision-making, marketing, verbal and written communication, strategic planning, networking, relationship building and management, conflict resolution, goal setting and performance, new business development, territory management, account management and devel

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/BUSINESS-DEVELOPMENT/37521676.pdf

In [23]:
query = "Do we have some senior sales profiles?"

ask_your_own_data(query, vector_store, gptmodel, topn=3)

Profile 1:

1. Senior Sales Assistant/Sales Support with experience in monitoring ads, providing high-quality customer service, and managing office operations.
2. Experienced Senior Sales Assistant/Sales Support with a track record of accurately monitoring and evaluating ads, providing exceptional customer service, and effectively solving customer challenges. Skilled in maintaining inventory, managing office operations, and creating presentations. 
3. No specific products or technologies mentioned in the resume.
4. Skills mentioned in the resume include attention to detail, customer service, database management, inventory management, office operations, presentations, quality control, sales, and phone handling.
5. The resume does not mention any specific company names or organizations.
6. The resume mentions Northwood High School as the educational background.
7. This profile is valuable due to the extensive experience in sales support and customer service, as well as the ability to eff

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/SALES/15273850.pdf


Profile 2:

1. Senior Account Manager with experience in selling E-commerce and Data Management Software and PVD materials to Fortune 500 semiconductor customers.
2. Experienced Senior Account Manager with a track record of success in selling E-commerce and Data Management Software as well as PVD materials to semiconductor industry. Grew Northwest territory from $300K/yr to $5 million/yr in 3 1/2 years. Certified in sales techniques and coaching. Skilled in team leadership, strategic planning, and relationship building.
3. E-commerce and Data Management Software, PVD materials, ERP systems (SAP, Oracle, Baan, JDEdwards)
4. Key decision making, team leadership, marketing, verbal and written communication, strategic planning, networking, relationship building and management, conflict resolution, goals and performance, new business development, territory management, account management, product and service sales.
5. Ariba (partnered with), LSI, Micron, STMicro, Philips, Hyundai, Motorola,

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/BUSINESS-DEVELOPMENT/37521676.pdf


Profile 3:

1. This profile has experience in sales and customer retention, with a focus on building networking sites to increase profitability.
2. The candidate has led sales calls and set goals for sales and customer retention. They have experience in various industries such as hospitality, logistics, healthcare, retail, and contractors. They have a strong track record of exceeding sales goals and maintaining high retention rates. They have also received awards for their performance and have experience in mortgage lending. They have a strong skill set in customer targeting, contract negotiations, prospecting, internet marketing, and leadership. They have a degree in Arts and Science.
3. Products or technologies mentioned in the resume include SaaS (Software as a Service) and websites.
4. Skills mentioned in the resume include customer targeting, contract negotiations, prospecting, internet marketing, leadership, conflict resolution techniques, hiring, training and supervision, opera

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/BUSINESS-DEVELOPMENT/18293620.pdf

In [24]:
query = "Show me some candidates with deep expertise in SQL Server"

ask_your_own_data(query, vector_store, gptmodel, topn=3)

Profile 1:

1. The candidate has skills in Microsoft Word, Excel, Access, SQL Server, VBA, and VisualStudio.
2. The candidate is proficient in using Microsoft Office applications such as Word, Excel, and Access. They also have experience with SQL Server, VBA programming, and VisualStudio.
3. Microsoft Word, Excel, Access, SQL Server, VBA, VisualStudio.
4. Microsoft Word, Excel, Access, SQL Server, VBA, VisualStudio.
5. No information provided in the resume.
6. No information provided in the resume.
7. The candidate's proficiency in Microsoft Office applications and programming languages such as VBA and VisualStudio make them valuable for tasks involving data analysis, database management, and software development.

⚠️This summary is generated by an AI (Azure Open AI). Examine and use carefully.

➡️Resume 1: cv/INFORMATION-TECHNOLOGY/20001721.pdf Page: 2 with confidence = 0.8610725


/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/INFORMATION-TECHNOLOGY/20001721.pdf


Profile 2:

1. The resume mentions experience with FACET, HEAT, SQL Server 2005, Cognos, ER/Studio, Mini-tab, Oracle, SAS, SharePoint Foundation 2007/2010, SPSS, and pcAnywhere.
2. The candidate has advanced user skills in Microsoft Office suite including Excel, Word, One Note, Outlook, Power Point, Project, SharePoint, and Visio.
3. Products or technologies mentioned in the resume include FACET, HEAT, SQL Server 2005, Cognos, ER/Studio, Mini-tab, Oracle, SAS, SharePoint Foundation 2007/2010, SPSS, and pcAnywhere.
4. Skills mentioned in the resume include advanced user skills in Microsoft Office suite including Excel, Word, One Note, Outlook, Power Point, Project, SharePoint, and Visio.
5. The resume does not mention any specific company names or organizations where the candidate has worked.
6. The resume does not mention any specific schools or universities.
7. The candidate's advanced user skills in Microsoft Office suite and experience with various software packages make them valua

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/ENGINEERING/32081266.pdf


Profile 3:

1. College graduate with a strong work ethic and focus on details, seeking an entry-level position as an insurance claims adjuster.
2. College graduate with a proven record of success in all endeavors, interested in insurance claims work. Experienced in creating SQL queries, stored procedures, and tables to extract data. Skilled in analyzing data to create executive team dashboards and daily visualizations. Proficient in utilizing Excel pivot tables and conducting data reviews to ensure data quality. Experienced in providing technical support and basic development for computer applications and programs. Developed and enhanced WebI reports and universes through Business Objects. Collaborated with team members and clients to gather HR/payroll report requirements.
3. SQL Server 2005, SQL Server 2008, SQL Server 2008R2, VM-Ware, ODBC, SQL, PL/SQL, Access, MS Office Suite, MS Visio, Oracle 10g, Oracle 11g, SAP Business Objects BI Suite, PeopleSoft, ADP, cView, SQL Server Studio

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/CONSULTANT/30863060.pdf

In [25]:
query = "Display some candidates with SCRUM skills"

ask_your_own_data(query, vector_store, gptmodel, topn=3)

Profile 1:

1. The candidate has skills in Adobe, Agile, SCRUM, analytical skills, social media, SEO, SEM, PR, pricing, product marketing, product testing, optimization, promotions, and writing proposals.
2. The candidate is proficient in Adobe software and has experience in Agile and SCRUM methodologies. They possess strong analytical skills and are knowledgeable in social media, SEO, SEM, and PR. They have expertise in pricing, product marketing, and product testing, with a focus on optimization and promotions. Additionally, they have experience in writing proposals.
3. Adobe, Agile, SCRUM, social media, SEO, SEM, PR
4. Analytical skills, pricing, product marketing, product testing, optimization, promotions, writing proposals
5. No specific companies or organizations mentioned in the resume
6. No specific schools or universities mentioned in the resume
7. The candidate's skills in Adobe, Agile, SCRUM, and various marketing and analytical areas make them valuable for roles in digital 

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/CONSULTANT/29770086.pdf


Profile 2:

1. This profile has experience in conducting functional and physical configuration audits to meet CMMI standards.
2. The candidate has worked as a Business Analyst and Scrum Master from October 2014 to March 2019. They facilitated requirements meetings and grooming sessions with the development team and clients to identify desired features and functionality. They also facilitated daily scrums, sprint reviews, sprint retrospectives, and sprint planning with the project team. They captured and translated requirements to the development team using Jira, Confluence, and during daily scrums. Additionally, they tested the application across different platforms to ensure proper implementation of requirements.
3. No specific products or technologies are mentioned in the resume.
4. The skills mentioned in the resume include functional and physical configuration audits, business analysis, Scrum Master, requirements gathering, Jira, Confluence, and testing.
5. The company name or org

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/INFORMATION-TECHNOLOGY/28697203.pdf


Profile 3:

1. The resume highlights skills in architecture, business process, consulting, contracts, client management, design and construction, engineering support, estimating, project management, research, and scheduling.
2. The candidate has experience in architecture and design, with a focus on business process improvement and consulting. They have worked on various projects involving client management, contracts, and engineering support. They are skilled in estimating project costs and managing project schedules. They have also conducted research and developed proposals for clients. Additionally, they have experience in the retail industry and have worked on projects related to office design and construction.
3. The resume does not mention any specific products or technologies.
4. The skills mentioned in the resume include architecture, business process, consulting, contracts, client management, design and construction, engineering support, estimating, project management, resear

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/CONSULTANT/30083884.pdf

In [26]:
query = "Display some profiles who has experience in PR campaigns"

ask_your_own_data(query, vector_store, gptmodel, topn=3)

Profile 1:

1. Public Relations intern with over 5 years of experience working for leading agencies such as Weber Shadwick, GolinHarris, and Latina PR, with strong experience in tech, B2B, social impact, and consumer products.
2. Public Relations intern with more than 5 years of experience working for leading agencies such as Weber Shadwick, GolinHarris, and Latina PR. Has a strong track record in tech, B2B, social impact, and consumer products. Experienced in working across multiple cross-office teams based in different continents. Proficient in Salesforce, Adobe Package (InDesign, Illustrator, Photoshop), MS Office, and cloud-based calendars. Has a diverse multicultural background with dual citizenship in America and Brazil, and has been educated in the San Francisco Bay Area and Switzerland.
3. Hootsuite, Sprout, TrackMaven
4. Salesforce, Adobe Package (InDesign, Illustrator, Photoshop), MS Office (Word, Excel, PowerPoint, Keynote), Cloud-based calendars
5. Weber Shadwick, GolinHarr

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/DIGITAL-MEDIA/31162167.pdf


Profile 2:

1. Marketing Executive with strong presentation, communication, and team management skills, seeking a high-level position in a professional corporate environment.
2. Marketing Executive with expertise in Hubspot/SEO, Marketo, Google Analytics/PPC, Wordpress, Salesforce.com, and project management. Accomplishments include implementing SCRUM methodology resulting in a 15% increase in productivity, creating metrics-based reporting for marketing and sales teams, spearheading a new-hire program to increase retention, and initiating a geo-local landing page PPC strategy resulting in a 35% lift in conversion. Experience includes roles as a Consultant, VP of Marketing, Director of Marketing, and CEO/Client Success Officer.
3. Hubspot, SEO, Marketo, Google Analytics, PPC, Wordpress, Salesforce.com
4. Project Management, Human Resources, Operations Management, Staff Development
5. CompanyName, City, State (Consultant Jan 2015 to Jan 2016), CompanyName, City, State (VP of Marketing O

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/CONSULTANT/29770086.pdf


Profile 3:

1. Social media manager with experience in marketing and public relations, strong communication skills, and proficiency in computer office programs.
2. Experienced social media manager with a background in marketing and public relations. Highly organized and able to multi-task. Strong communicator with the ability to speak and write well. Proficient in computer office programs. Skilled in developing and nurturing relationships with clients and team members. Works well under pressure and is an effective problem solver. Experience includes managing social media profiles, creating strategic plans for social media posts, and assisting with office duties.
3. Microsoft programs, Facebook, Twitter, Instagram, Constant Contact
4. Communication, organization, multitasking, problem-solving, social media management
5. CompanyName
6. University of Louisiana at Lafayette
7. This profile is valuable due to the candidate's experience in social media management, marketing, and public rela

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/DIGITAL-MEDIA/11270462.pdf

In [27]:
query = "Any director of software development?"

ask_your_own_data(query, vector_store, gptmodel, topn=3)

Profile 1:

1. A highly skilled Senior Software Executive with expertise in strategic planning, product development, and agile software development.
2. Over the past 15 years, this candidate has reported directly to C-level executives and Board members, successfully bridging the gap between business and engineering. They have implemented strategic plans, aligned engineering teams to business goals, and reorganized and transitioned teams to become smooth running agile groups. They have managed the entire software development process and employees, including QA, project management, technical support, and on and offshore teams. They have also successfully onboarded engineers and technology from acquired companies. They have managed globally distributed teams and have been an early adopter of critical trends in methodologies and practices.
3. Java, Amazon AWS, Hibernate, PHP, EC2, Elastic Search, C++, JSMVC, JUnit, C, HTML, Selenium, CanJS, CSS, Aurora, PL/SQL, Bootstrap, Jenkins, Oracle, 

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/ENGINEERING/10624813.pdf


Profile 2:

1. Director of Information Technology with experience in C#, Oracle SQL, SharePoint, ASP.NET, PL/SQL, and project management.
2. Experienced Director of Information Technology with a strong background in C#, Oracle SQL, SharePoint, and ASP.NET. Skilled in database design and administration, application development, team leadership, and IT strategy. Successfully led projects and provided business analysis to enhance workflows. Proficient in software development methodologies and project management. Previous experience includes roles as a Team Leader and Chief Information Officer, where I provided network and application support for large organizations.
3. C#, Oracle SQL, SharePoint, ASP.NET, PL/SQL, SQL Server, SQL*Plus, TFS, SharePoint Designer, IIS, ADO.NET, Microsoft SQL Server, XML, Tableau, DSCI.
4. C#, Oracle SQL, SharePoint, ASP.NET, PL/SQL, SQL Server, SQL*Plus, TFS, SharePoint Designer, Database Design, Database Administration, Data Management, Team Leadership, IIS

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/INFORMATION-TECHNOLOGY/13385306.pdf


Profile 3:

1. Highly accomplished Director of Information Technology with 20+ years of experience in software development, infrastructure design, and strategic planning.
2. Experienced IT executive with a track record of success in managing software development and infrastructure projects, overseeing daily operations, and aligning efforts with clients and corporate needs. Skilled in change management, collaboration, and mentoring. Accomplishments include system conversion, standardization of software and hardware infrastructure, and implementation of ERP and telecom systems.
3. Products or technologies mentioned in the resume: Windows 7, Office 2010, IP hybrid telecom system, legacy servers, virtualization.
4. Skills mentioned in the resume: Innovative leadership, change management, infrastructure design, strategic planning, technology development, collaboration and liaison, P&L oversight, organizational development, mentoring and coaching.
5. Companies or organizations where this pr

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/INFORMATION-TECHNOLOGY/28672970.pdf

In [28]:
query = "Any specialist in digital media, Adobe Creative?"

ask_your_own_data(query, vector_store, gptmodel, topn=3)

Profile 1:

1. Highly creative Graphic Designer with over ten years of experience in print packaging design, advertising, marketing, and brand development.
2. Experienced Graphic Designer proficient in Adobe Creative Suite, Esko Automation Engine Print Workflow Management System, Microsoft Office Suite, commercial printing processes, digital photography, project tracking and collaboration software, troubleshooting hardware/software issues, and network management. Currently working as a Digital Premedia Specialist, responsible for developing press-ready files in a commercial printing environment, conducting preflight checks on customer files, guiding customer press check tours, collaborating with press room staff to improve deliverables and reduce waste, and maintaining digital asset and artwork database.
3. Adobe Creative Suite (Photoshop, Illustrator, InDesign, Dreamweaver, Fireworks & Flash, Acrobat & Distiller), Esko Automation Engine Print Workflow Management System, Microsoft Offi

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/DIGITAL-MEDIA/27921561.pdf


Profile 2:

1. A hardworking, focused and determined artist with experience in graphic design and multimedia design.
2. This candidate has experience as a graphic designer, working on branding, logo design, and promotional materials for various companies. They have also worked as a freelance graphic designer, creating graphics for pamphlets, banners, and web graphics. They have a Bachelor's degree in Digital Art & Design.
3. Adobe Photoshop, Adobe After Effects, Adobe Illustrator, Adobe InDesign, Adobe Premiere, Adobe Dreamweaver, Sketchbook Pro, Intermediate Html Coding, Microsoft Word, Windows, Troubleshooting, Calculator, printer, fax machine, Microsoft PowerPoint.
4. Graphic design, branding, logo design, poster design, flyer design, digital design.
5. CompanyName, City, State (multiple companies).
6. Towson University, City, State, United States, Digital Art & Design Community College of Baltimore County, City, State, United States.
7. This candidate has won a design competition 

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/DESIGNER/29147100.pdf


Profile 3:

1. Marketing specialist and graphic designer with expertise in graphic design, corporate marketing communications, project and event management, and print production management.
2. Accomplished marketing professional with a proven track record in graphic design, corporate marketing communications, and project management. Recognized for multitasking abilities, attention to detail, and high quality standards. Experienced in managing corporate brand strategy, designing marketing communications, and coordinating events. Skilled in Adobe Creative Suites, Microsoft Office programs, and print production management. Seeking a creative position to utilize creative abilities and business acumen.
3. Adobe Photoshop, Illustrator, and InDesign, Microsoft Office programs (Powerpoint, Word, Excel)
4. Art Direction, Corporate Brand Strategy, Graphic Design, Corporate Marketing Communications, Creative Print and Digital Content Development, Event/Trade Show Management, Print Production Man

/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/RAG - Resume analysis usecase/cv/DESIGNER/15425154.pdf

## 4. Gradio webapp

Let's build a quick webapp with gradio to test our RAG application

In [29]:
def cv_rag_function(query):
    """
    RAG function with Azure Cognitive Search and Azure Open AI
    Input: query
    Output: result list with the results for a gradio webapp
    """
    # Calling Azure Cognitive Search
    results = vector_store.similarity_search_with_relevance_scores(
        query=query,
        k=5,
        score_threshold=0.5,
    )

    msg = "\n\n[Note] This summary is generated by an AI (Azure Open AI). Examine and use carefully."

    for idx in range(3):
        # Lists declaration
        results_list = []
        source_list = []
        confidence_list = []
        
        # Output generation
        for idx in range(3):
            # Get Azure Cognitive Search result
            reference = results[idx][0]
            ref = reference.__dict__
            ref = ref["page_content"]
            answer = azure_openai(ref, gptmodel) + msg

            # Get source information
            source = reference.metadata
            doc_source = source["source"]
            # Get page
            page_source = source["page"] + 1
            # Get full source reference
            source_ref = f"Source: {doc_source} Page: {page_source}"
            source_list.append(source_ref)
            # Get confidence
            confidence = results[idx][1]
            confidence_list.append(confidence)

            # Adding all the results into a single list
            results_list.append(answer)
            results_list.append(source_list[idx])
            results_list.append(confidence_list[idx])
            
    return results_list

### Webapp function definition

In [34]:
image_url = "https://github.com/retkowsky/images/blob/master/HR.jpg?raw=true"
image = "<center> <img src= {} width=500px></center>".format(image_url)
header = "RAG application with Azure Open AI & Azure Cognitive Search (Resume analysis example)"

questions = [
    "Do we have some designer profiles with some great international experiences",
    "I need some business development profiles with more than 10 years of experience",
    "Do we have some senior sales profiles?",
    "Show me some candidates with deep expertise in SQL Server",
    "Display some candidates with SCRUM skills",
    "Display some profiles who has experience in PR campaigns",
    "Any director of software development?",
    "Any specialist in digital media, Adobe Creative?",
]

inputs = gr.Text(type="text", label="Question:", lines=5, max_lines=10)

outputs = [
    gr.Text(label="Candidate 1 - Azure Open AI summary"),
    gr.Text(label="Resume source"),
    gr.Text(label="Confidence"),
    gr.Text(label="Candidate 2 - Azure Open AI summary"),
    gr.Text(label="Resume source"),
    gr.Text(label="Confidence"),
    gr.Text(label="Candidate 3 - Azure Open AI summary"),
    gr.Text(label="Resume source"),
    gr.Text(label="Confidence"),
]

theme = "freddyaboulton/test-blue"
# https://huggingface.co/spaces/gradio/theme-gallery

cv_rag_webapp = gr.Interface(
    fn=cv_rag_function,
    inputs=inputs,
    outputs=outputs,
    description=image,
    title=header,
    examples=questions,
    theme=theme,
)

### Running the webapp

In [35]:
cv_rag_webapp.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://88315a9b520b52e239.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


## 5. Post Processing

We can delete our index if needed

In [32]:
index_status(index_name, azure_cs_endpoint, azure_cs_key)

Azure Cognitive Search Index: resume-rag-demo 

{
     "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#indexes/$entity",
     "@odata.etag": "\"0x8DBE6962627FA4D\"",
     "name": "resume-rag-demo",
     "defaultScoringProfile": null,
     "fields": [
          {
               "name": "id",
               "type": "Edm.String",
               "searchable": false,
               "filterable": true,
               "retrievable": true,
               "sortable": false,
               "facetable": false,
               "key": true,
               "indexAnalyzer": null,
               "searchAnalyzer": null,
               "analyzer": null,
               "normalizer": null,
               "synonymMaps": []
          },
          {
               "name": "content",
               "type": "Edm.String",
               "searchable": true,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
               "facet

In [33]:
document_count, storage_size = index_stats(index_name, azure_cs_endpoint, azure_cs_key)

Azure Cognitive Search index status for: resume-rag-demo 

{
  "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#Microsoft.Azure.Search.V2021_04_30_Preview.IndexStatistics",
  "documentCount": 1509,
  "storageSize": 46394403
}


In [ ]:
#delete_index(index_name, azure_cs_endpoint, azure_cs_key)

> End